# --- Day 10: Balance Bots ---

http://adventofcode.com/2016/day/10

You come upon a factory in which many robots are zooming around handing small microchips to each other.

Upon closer examination, you notice that each bot only proceeds when it has two microchips, and once it does, it gives each one to a different bot or puts it in a marked "output" bin. Sometimes, bots take microchips from "input" bins, too.

Inspecting one of the microchips, it seems like they each contain a single number; the bots must use some logic to decide what to do with each chip. You access the local control computer and download the bots' instructions (your puzzle input).

Some of the instructions specify that a specific-valued microchip should be given to a specific bot; the rest of the instructions indicate what a given bot should do with its lower-value or higher-value chip.

For example, consider the following instructions:

```
value 5 goes to bot 2
bot 2 gives low to bot 1 and high to bot 0
value 3 goes to bot 1
bot 1 gives low to output 1 and high to bot 0
bot 0 gives low to output 2 and high to output 0
value 2 goes to bot 2
```

Initially, bot 1 starts with a value-3 chip, and bot 2 starts with a value-2 chip and a value-5 chip.
Because bot 2 has two microchips, it gives its lower one (2) to bot 1 and its higher one (5) to bot 0.
Then, bot 1 has two microchips; it puts the value-2 chip in output 1 and gives the value-3 chip to bot 0.
Finally, bot 0 has two microchips; it puts the 3 in output 2 and the 5 in output 0.
In the end, output bin 0 contains a value-5 microchip, output bin 1 contains a value-2 microchip, and output bin 2 contains a value-3 microchip. In this configuration, bot number 2 is responsible for comparing value-5 microchips with value-2 microchips.

Based on your instructions, **what is the number of the bot that is responsible for comparing value-61 microchips with value-17 microchips?**

In [206]:
with open(f'inputs/10.txt') as f:
    data = f.read().strip().split("\n")
print("Length:", len(data))
data[:10]

Length: 231


['bot 88 gives low to bot 51 and high to bot 42',
 'bot 13 gives low to bot 4 and high to bot 167',
 'bot 90 gives low to bot 78 and high to bot 199',
 'bot 84 gives low to bot 205 and high to bot 201',
 'bot 41 gives low to bot 48 and high to bot 15',
 'bot 15 gives low to bot 156 and high to bot 54',
 'bot 70 gives low to output 10 and high to bot 4',
 'bot 140 gives low to bot 206 and high to bot 189',
 'value 67 goes to bot 187',
 'bot 124 gives low to bot 99 and high to bot 102']

In [10]:
import regex
from collections import defaultdict

In [134]:
def check_bot(bot, x=17, y=61):
    """return True and prints bot name if it contains nums x and y, returns False otherwise"""
    
    if len(bots[bot]) < 2: return False
    
    if x in bots[bot] and y in bots[bot]:
        print(bot, "contains", bots[bot])
        return True
    else:
        return False
bots['bot 71'] = [17, 61]
check_bot('bot 71')

bot 71 contains [17, 61]


True

In [172]:
def step(line):
    """performs the instructions passed in"""
    
    if line.startswith("bot"):
        p = r'(bot \d+) gives low to (\w+ \d+) and high to (\w+ \d+)'
        giving_bot, low_bot, high_bot = regex.findall(p, line)[0]
        
        # don't do the instruction if the bot doesn't have 2 chips
        if len(bots[giving_bot]) < 2:
            return False
        
        # checking only the giving bot as thats the one which actually does the comparision
        # this could fail if the bot comparing is never called to do so and just holds on the to the chip 
        check_bot(giving_bot)
        
        low_val = min(bots[giving_bot])
        bots[giving_bot].remove(low_val)
        bots[low_bot].append(low_val)
        
        high_val = max(bots[giving_bot])
        bots[giving_bot].remove(high_val)
        bots[high_bot].append(high_val)
        
    elif line.startswith("value"):
        p = r'value (\d+) goes to (bot \d+)'
        val, bot = regex.findall(p, line)[0]
        bots[bot].append(int(val))
        
    else:
        print(f"can't deal with: {line}")
    
    return True

Running the instructions in one loop didn't work, so I'm going to keep running them until stuff happens:

In [192]:
bots = defaultdict(list) # well bots and the output buffers both

with open(f'inputs/10.txt') as f:
    data = f.read().strip().split("\n")

i = 0

# we keep looping through the instructions until they are all executed
while len(data) > 0: 
    i = (i+1) % len(data)
    if step(data[i]):
        del data[i]

bot 27 contains [61, 17]


# --- Part Two ---

What do you get if you multiply together the values of one chip in each of outputs 0, 1, and 2?

---

Grabbing the first three outputs, assuming they all have only one number:

In [204]:
outputs = [bots[f"output {p}"][0] for p in range(3)]
outputs

[7, 53, 37]

Multiplying them together:

In [202]:
import numpy as np
np.prod(outputs)

13727

I've always wanted to use reduce more, despite [what this guys says about it](https://www.artima.com/weblogs/viewpost.jsp?thread=98196), so here goes:

In [203]:
from functools import reduce
reduce(lambda x, y: x*y, outputs)

13727

`13727`

# Notes

- this was an interesting problem, becuase it wasn't clear what to do.